In [2]:
import yfinance as yf
import pandas as pd
import zipfile
import os

In [3]:
base_dir = "data"

def fetch_tickers_from_wikipedia(url, table_index, column_name):
    """Scrape tickers from a Wikipedia table."""
    try:
        tables = pd.read_html(url)
        return tables[table_index][column_name].dropna().tolist()
    except Exception as e:
        print(f"Error fetching tickers from {url}: {e}")
        return []

indices = {
    #"ftse100": 
    
    # {"url": "https://en.wikipedia.org/wiki/FTSE_100_Index",
    #     "table_index": 4,
    #     "column": "Ticker",
    #     "suffix": ".L"
    # },
    # "nikkei225": {
    #     "url": "https://en.wikipedia.org/wiki/Nikkei_225",
    #     "table_index": 1,
    #     "column": "Ticker",
    #     "suffix": ".T"
    # }
    # "sp500": {
    #     "url": "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    #     "table_index": 0,
    #     "column": "Symbol",
    #     "suffix": ""
    # },

    'sp500_old': {
        "url": "https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=360899452",
        "table_index": 0,
        "column": "Ticker symbol",
        "suffix": ""
    },    
}

for index, meta in indices.items():
    print(f"\n Downloading data for: {index.upper()}")
    tickers = fetch_tickers_from_wikipedia(meta["url"], meta["table_index"], meta["column"])
    # tickers = pd.read_csv(r'C:\\physics_masters\\nikkei_stock_average_weight_en.csv', encoding='utf-8')['Code'].tolist()
    tickers = [str(t).strip() for t in tickers]
    print(tickers)
    index_dir = os.path.join(base_dir, index)
    os.makedirs(index_dir, exist_ok=True)

    for ticker in tickers:
        full_ticker = ticker + meta["suffix"]
        try:
            df = yf.download(full_ticker, start="2000-01-01", end="2010-04-26", interval="1d")
            if not df.empty:
                df.to_csv(os.path.join(index_dir, f"{ticker}.csv"))
                print(f"Saved: {ticker}")
        except Exception as e:
            print(f"❌ {ticker}: {e}")


['MMM', 'ABT', 'ANF', 'ADBE', 'AMD', 'AES', 'AET', 'AFL', 'A', 'APD', 'AKAM', 'AKS', 'AA', 'AYE', 'ATI', 'AGN', 'ALL', 'ALTR', 'MO', 'AMZN', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AMP', 'ABC', 'AMGN', 'APC', 'ADI', 'AON', 'APA', 'AIV', 'APOL', 'AAPL', 'AMAT', 'ADM', 'ASH', 'AIZ', 'T', 'ADSK', 'ADP', 'AN', 'AZO', 'AVB', 'AVY', 'AVP', 'BHI', 'BLL', 'BAC', 'BK', 'BCR', 'BAX', 'BBT', 'BDX', 'BBBY', 'BMS', 'BRK.B', 'BBY', 'BIG', 'BIIB', 'HRB', 'BMC', 'BA', 'BXP', 'BSX', 'BMY', 'BRCM', 'BF.B', 'CHRW', 'CA', 'COG', 'CAM', 'CPB', 'COF', 'CAH', 'CFN', 'CCL', 'CAT', 'CBG', 'CBS', 'CELG', 'CNP', 'CTL', 'CEPH', 'CERN', 'CF', 'SCHW', 'CHK', 'CVX', 'CB', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CTXS', 'CLF', 'CLX', 'CME', 'CMS', 'COH', 'KO', 'CCE', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CSC', 'CPWR', 'CAG', 'COP', 'CNX', 'ED', 'STZ', 'CEG', 'CBE', 'GLW', 'COST', 'CVH', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DF', 'DE', 'DELL', 'DNR', 'XRAY', 'DVN', 'DV', 'DO', 'DTV', 'DFS', 'DISCA', 'D', 'RRD', 'DOV', 'D

[*********************100%***********************]  1 of 1 completed


Saved: MMM


[*********************100%***********************]  1 of 1 completed


Saved: ABT


...

Saved: XRX


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['XLNX']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['XL']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['XTO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2010-04-26)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['YHOO']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed


Saved: YUM


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ZMH']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2010-04-26)')
[*********************100%***********************]  1 of 1 completed

Saved: ZION
